<a href="https://colab.research.google.com/github/ManishPrasad0511/SoME/blob/main/Animation_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  The Summer of Math Exposition 2022

This notebook has the code for the thumbnail, and the outro animation 12:03 to 13:35 in my video:
https://youtu.be/2-iuxnpJFKU

This code only works for N greater than or equal to 2.

To run this is google colab, first uncomment the following cell and then run it. Next, click on 'Restart and run all' in 'Runtime' option at the top. You could also just click 'Restart runtime' if you want to run the cells individually to understand them. 

In [ ]:
# Uncomment the following line if running in Google Colab
''' !sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install IPython --upgrade  
!pip install manim '''

The version I used to make my video is: Manim Community v0.16.0.post0

In [2]:
from manim import *
import numpy as np
import itertools

Manim Community v0.16.0.post0

Choose N>=2\
You can make your own animations by playing around with the following parameters. But manim has fixed dimensions of the screen and it is 14.2 units horizontally and 8 units vertially. So make sure that you choose a and b appropriately otherwise the image doesn't fit in the screen. And you always the scale down your units using appropriate ratios so that it fits in the screen.

In [3]:
a = 8                   # length
b = 6                   # breadth
sides = [a,b]
N = 7                   # Value of N (see the question) CHOOSE N>=2
p1 = [-2,1]             # coordinate of 1st point
p2 = [0,-2]             # coordinate of 2nd point
d = 2

The following cell gives the coordinates of all the 8 solutions in the outer grid.

In [4]:
signs  = list(itertools.product([1, -1], repeat=d))
final_pts = []
for i in signs:
  shifted_p2 = p2 + 2*(N-1)*np.multiply(i,sides)
  for j in range(d):
    final_shift = np.zeros(d)
    final_shift[j] = np.multiply(i,2*(sides[j]/2 - i[j]*(p2[j])))[j]
    final_pts.append(shifted_p2 + final_shift)

The array final_coords_all at the end of this code has the coordinates of all the reflection points on the walls for all the 8 solutions.\
final_coords_all = [ [array of all reflection points for 1st outer coordinate] , [2nd] , .  . [8th] ]\
array of all reflection points for 1st outer coordinate = [ [x1,y1] , [x2,y2] . .[x8,y8] ]

In [8]:
final_coords_all = []
final_coord_out = []
for p in final_pts:

  x_inters = abs(p[0])//(sides[0]/2) - abs(p[0])//sides[0]
  y_inters = abs(p[1])//(sides[1]/2) - abs(p[1])//sides[1]
  m = (p[1]-p1[1]) / (p[0]-p1[0])

  vert_walls = []
  horz_walls = []
  if p[0]>=0:
    x_start = sides[0]/2
  else:
    x_start = -sides[0]/2
  if p[1]>0:
    y_start = sides[1]/2
  else:
    y_start = -sides[1]/2

  for i in np.arange(x_inters):
    y = p1[1] + m*(x_start-p1[0])
    vert_walls.append([x_start,y])
    x_start = x_start + np.sign(p[0])*sides[0]

  for i in np.arange(y_inters):
    x = p1[0] + (y_start-p1[1])/m
    horz_walls.append([x,y_start])
    y_start = y_start + np.sign(p[1])*sides[1]

  all_inters = np.array(vert_walls+horz_walls)
  indexlist =np.argsort(np.linalg.norm(all_inters,axis=1))
  all_inters = all_inters[indexlist]
  final_coord_out.append(all_inters)

  grid_dim = []
  grid_coords = []
  final_coords = []
  for i in range(len(all_inters)):
    all_inters[i] += np.array(sides)/2
    grid_dim.append([all_inters[i][0]%sides[0] , all_inters[i][1]%sides[1]])
    grid_coords.append([all_inters[i][0]//sides[0] , all_inters[i][1]//sides[1]])
    single_final_coord=np.zeros(d)
    for j in range(d):
      if grid_coords[i][j]%2 == 0:
        single_final_coord[j] = (grid_dim[i][j]) - sides[j]/2
      else:
        single_final_coord[j] = sides[j] - (grid_dim[i][j]) - sides[j]/2
    
    final_coords.append(single_final_coord)

  final_coords_all.append(final_coords)
  

To make my thumbnail, I chose N=7. And instead of animating the whole thing, I just the self.add() command to create an image since I only need an image.

In [ ]:
%%manim thumbnail
class thumbnail(Scene):
    def construct(self):
      rect = Rectangle(width=sides[0], height=sides[1])

      d1 = Dot(point=np.array([p1[0],p1[1], 0.]) , color=BLUE)
      d6 = Dot(point=np.array([p2[0], p2[1], 0.]) , color=BLUE)
      
      bruh = [final_coords_all[5]]
      for i in range(len(bruh)):
        dots = [d1]
        for j in range(len(bruh[i])):
          dots.append(Dot(point=np.array([bruh[i][j][0],bruh[i][j][1], 0.]) , color=BLUE))
          line = Line(start=dots[-2].get_center() , end=dots[-1].get_center())
          self.add(line)
        line = Line(start=dots[-1].get_center() , end=d6.get_center())
        self.add(line)
      self.add(rect,d1,d6)


The following cell is important for making the animation. In Manim, for shifting one point to another, we need the difference between the coordinates. So I converted the array final_coords_all to an array of all the "shifts" I need to make my animation.

In [10]:
for i in range(len(final_coords_all)):
  for j in range(len(final_coords_all[i])-1,-1,-1):
    if j==len(final_coords_all[i])-1:
      final_coords_all[i].append( np.array(p2) - final_coords_all[i][j])
    if j==0:
      final_coords_all[i][j] -= np.array(p1)
    else:
      final_coords_all[i][j] -=  final_coords_all[i][j-1]

This cell is for making the animation. Instead of doing making the previous cell, I could have incorporated that whole thing in the following cell and it would definitely be a better way to do it since it wouldn't destroy what the array final_coords_all was before. You can work this out if you want to.

In [ ]:
%%manim rect_mirror
class rect_mirror(Scene):
    def construct(self):
      rect = Rectangle(width=sides[0], height=sides[1])

      dot = Dot(point=np.array([p1[0],p1[1], 0.]) , color=BLUE)
      d6 = Dot(point=np.array([p2[0], p2[1], 0.]) , color=BLUE)
      
      def update_path(path):
            previous_path = path.copy()
            previous_path.add_points_as_corners([dot.get_center()])
            path.become(previous_path)
      
      self.add(rect)
      self.wait()
      self.add(dot,d6)
      self.wait()
      
      for i in range(len(final_coords_all)):
        dot = Dot(point=np.array([p1[0],p1[1], 0.]) , color=BLUE)

        path = VMobject()     
        path.set_points_as_corners([dot.get_center(), dot.get_center()])
        path.add_updater(update_path)
        
        self.add(path, dot)
        dot.generate_target()
        for j in range(len(final_coords_all[i])):
          dot.target.shift(final_coords_all[i][j][0]*RIGHT,final_coords_all[i][j][1]*UP)
          self.play(MoveToTarget(dot))

        self.play(FadeOut(dot,path))
        self.remove(dot,path)